# Additional Parameters Influencing Human Strikezones

## Goals

The objective of this project is to determine what factors other than position influence an umpire's accuracy when calling balls and strikes. I plan to look at factors such as inning and batter's handedness that should not influence the data but possibly does as well as factors such as batter's height that should influence ball/strike calls.

## Methods

I have created multiple models based on a pitch's x and y coordinates as they cross the plate in order to set up a baseline for accuracy. I then will predict data using this model and find the subsets that are more or less accurate. Finally, I will add the factors that most directly influence the data as explanatory variables for a final model.

## The data

The data is pitch data that comes straight from MLB and has been collected into nice csv files on kaggle (https://www.kaggle.com/pschale/mlb-pitch-data-20152018). Pitches included are from the 2015-2018 seasons. Additionally this dataset includes data that can be joined with the pitch data to get information about the atbat (ie inning, batter, pitcher) and game (ie umpire, weather). 


In [52]:
#Data importing and manipulating, mostly from step 1
import pandas as pd
import numpy as np
import pickle
import os.path
games = pd.read_csv("games.csv")
atbats= pd.read_csv("atbats.csv")
players = pd.read_csv("player_names.csv")
pitches_full = pd.read_csv("pitches2.csv")
pitches = pitches_full
pitch = pitches[['px', 'pz', 'code']][pitches['code'].isin(["C", "B", "*B"])] #only balls and called strikes


#These two rows are most important for step 2, as they clean the data and transform one column.
pitch = pitches[['px', 'pz', 'code', 'ab_id']][pitches['code'].isin(["C", "B"])].dropna(axis='rows', how='any') #drop NA's
pitch['call'] = pitches['code'].isin(["C"]) #Call will be True for strikes, False for balls


strikes = pitch[['px', 'pz']][pitches['code'].isin(["C"])] #We use these later for graphical summaries
balls = pitch[['px', 'pz']][pitches['code'].isin(["B"])]

#Pickle is a great package that stores python variables as files. This is really useful as it saves me from running
#every model every time I want to run this document. It also allows me to share the models online so others can view the results immediately.
use_pickle = True

c:\users\tenor\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\users\tenor\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


# Baseline Models

Using only the positional data as the ball crosses the plate, I created four models to predict whether the pitch would be called a ball or a strike. The first, a logistic regression, had an underwhelming performance. The second, a nearest neighbors model, performed quite well. This performance was matched by both of the nerual networks I trained. 

After evaluating these four models, I took the confidences of one neural network and the nearest neighbors model and averaged them to create a final model, although this step similarly did not improve accuracy.

Given that many of these models have topped out at around a 91.5% accuracy, I infer that given these explanatory paramters, this level of accuracy is near the limit we can get due to variance in umpire calls.

## Logistic Regression

### Process:
I began with a simple logistic regression. It was a simple, although relatively unsuccessful starting point. I first tried giving it px and pz values, but the model just predicted everything to be a ball. However, when giving it distance from the center of the observed values it was able to find success.

### Results:
In the end I was able to create a moderately successful classifier with 88.7% accuracy.

### Remaining Questions:
There is no guarentee the mean of the px and pz values is the best for this regression. It would be possible to try measuring distance from a variety of points and seeing which would perform the best, but I belive other models will outperform even the most optimized logistic regression for this problem.

In [53]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model as lm
#My first attempt at a regression, which found no success
in_full = pitch[['px', 'pz']].values
out_full = pitch['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

#Now we train the model
model = lm.LogisticRegression()
model.fit(in_train, out_train)

score = model.score(in_test, out_test)
print("Score: " + str(score))
#That's just the proportion of balls. Not good.

c:\users\tenor\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Score: 0.6659111435017453


In [54]:
#This regression found decent success!

#I use distance from the center of our observations as our input metric.
#Distances have had greater success than pure px and pz values
#And by measuring distance from the mean we should get distance from roughly the center of the strike zone. 
mean_x = np.mean(pitch['px'])
mean_z = np.mean(pitch['pz'])
pitch['x_dist'] = pitch['px'] - mean_x
pitch['z_dist'] = pitch['pz'] - mean_z
pitch['distance'] = np.sqrt(pitch['x_dist'] * pitch['x_dist'] + pitch['z_dist'] * pitch['z_dist'])
#We split our data into training and test sets

in_full = pitch[['distance']].values
out_full = pitch['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)


if not use_pickle or not os.path.isfile('lm.pkl'):
    #Now we train the model
    linear_model = lm.LogisticRegression()
    linear_model.fit(in_train, out_train)
    if use_pickle:
        f=open('lm.pkl', 'wb')
        pickle.dump(linear_model, f)
        f.close()
else:
    f=open('lm.pkl', 'rb')
    linear_model = pickle.load(f, encoding='latin1')
    f.close()
    
score = linear_model.score(in_test, out_test)
print("Score: " + str(score))
print("88.7% accuracy is not terrible!")

Score: 0.8863611795925057
88.7% accuracy is not terrible!


## K Nearest Neighbors

### Process:
As an attempt to create a better model than the logistic regression, I decided to use a K nearest neighbors model. Immediately I was getting success above 90%, but to maximize the model's accuracy I looped through different values for n_neighbors to find the optimal one, which ended up being around 95. I also tried weighting based on distance, which had minimal impact on the model. 

### Results:
The end models were extremely successful with 91.5% for the unweighted model and 91.2% for the distance weighted model.
This is quite good!

### Remaining Questions:
I believe if I spent as much time tweaking the distance weighted model as I did the non-weighted model it could perform as well or better. I also would be interested if there are any meaningful differences in the confidences of these two models
The first model is a simple fraction over 95, but I would believe this one could be different.

In [55]:
#Here's where we will do the much more successful K-nearest neighbors model
from sklearn.neighbors import KNeighborsClassifier

In [56]:
#We split our data into training and test sets again
in_full = pitch[['px', 'pz']].values
out_full = pitch['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

I reccomend not running the following code block due to runtime. It creates 50  models with varying n_neigbors values to find the optimal one.

In [57]:
#Lest's find the optimal number of neighbors
#In the end I had the best results with n=95
#This code block takes a long time to run, I would reccomend ignoring it and just running the next one with the actual model.
if(False):
    knn_list_50s = []
    def KNN(n):
        classifier = KNeighborsClassifier(n_neighbors = n)
        classifier.fit(in_train, out_train)

        #Predicting, this takes a while
        test_predictions = classifier.predict(in_test)
        return np.sum(test_predictions == out_test) / len(test_predictions)

    for i in range(1, 50):
        knn_list_50s.append(KNN(i * 5))
    knn_list_50s

In [58]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 95)
knn.fit(in_train, out_train)

#Predicting, this takes a while
test_predictions = knn.predict(in_test)
    
print(str(np.sum(test_predictions == out_test) / len(test_predictions)))
#91.5% Accuracy, pretty good.

0.9145154597474693


In [59]:
knn_distance = KNeighborsClassifier(n_neighbors = 95, weights="distance")
knn_distance.fit(in_train, out_train)

#Predicting, this takes a while
test_predictions = knn_distance.predict(in_test)
#Predicting probabilities, I should do this at the same time as above, but I don't because I'm dumb.
probabilities = knn_distance.predict_proba(in_test)
print(str(np.sum(test_predictions == out_test) / len(test_predictions)))
#91.2% Accuracy 
#Not bad. I believe if I spent as much time tweaking this as I did the non-weighted model it could perform as well or better.
#I also would be interested if there are any meaningful differences in the confidences of these two models
#The first model is a simple fraction over 95, but I would believe this one could be different.

0.9122035588628098


## Neural Networks

### Process:
For my neural nets I used relu as the activation function. The documentation for this package reccomended using the 'adam' solver for larger datasets, so I utilized that. I trained a net with two hidden layers with 64 neurons each as well as a larger one with three hidden layers of size 128.

### Results:
Both of these models peformed similary to eachother and to the KNN model. This leads me to believe that, given this data set and these parameters, the best we may be able to do is around 91.5%.

As one would expect, these take a while to train, so use of the pickle package is highly reccomended here.

In [60]:
#Neural Network time!
import os.path
from sklearn.neural_network import MLPClassifier
if not use_pickle or not os.path.isfile('mlp_small.pkl'):
    mean_x = np.mean(pitch['px'])
    mean_z = np.mean(pitch['pz'])
    #Interestingly, although in the end relatively unimportant, the standard deviation of this data ends up being around 1
    #Therefore the z scores used here and the distances in the regression example are extremly similar
    pitch['z_px'] = (pitch['px'] - mean_x)/pitch.px.std(ddof=0) 
    pitch['z_pz'] = (pitch['pz'] - mean_z)/pitch.pz.std(ddof=0)

    #We split our data into training and test sets again (again)
    in_full = pitch[['z_px', 'z_pz']].values
    out_full = pitch['call']

    in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

    #adam is reccomended for datasets with samples over 1000s
    #early stopping will set aside a validation set and stop when that is no longer improving
    mlp = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(64, 64), random_state=1, early_stopping = True)

    mlp.fit(in_train, out_train)
    if use_pickle:
        f=open('mlp_small.pkl', 'wb')
        pickle.dump(mlp,f)
        f.close()
else:
    f=open('mlp_small.pkl', 'rb')
    mlp = pickle.load(f, encoding='latin1')
    f.close()

print(np.sum(mlp.predict(in_test) == out_test) / len(out_test))

0.6054390693814358


In [61]:
#Neural net 2, more hidden layers this time because why not?
if not use_pickle or not os.path.isfile('mlp_large.pkl'):
    mean_x = np.mean(pitch['px'])
    mean_z = np.mean(pitch['pz'])
    #Interestingly, although in the end relatively unimportant, the standard deviation of this data ends up being around 1
    #Therefore the z scores used here and the distances in the regression example are extremly similar
    pitch['z_px'] = (pitch['px'] - mean_x)/pitch.px.std(ddof=0) 
    pitch['z_pz'] = (pitch['pz'] - mean_z)/pitch.pz.std(ddof=0)

    #We split our data into training and test sets again (again)
    in_full = pitch[['z_px', 'z_pz']].values
    out_full = pitch['call']

    in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

    #adam is reccomended for datasets with samples over 1000s
    #early stopping will set aside a validation set and stop when that is no longer improving
    mlp_large = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(128, 128, 128), random_state=1, early_stopping = True)

    mlp_large.fit(in_train, out_train)
    if use_pickle:
        f=open('mlp_large.pkl', 'wb')
        pickle.dump(mlp_large, f)
        f.close()
else:
    f=open('mlp_large.pkl', 'rb')
    mlp_large = pickle.load(f, encoding='latin1')
    f.close()

print(np.sum(mlp_large.predict(in_test) == out_test) / len(out_test))

0.6080438807854186


## Combining the models

In [64]:
mean_x = np.mean(pitch['px'])
mean_z = np.mean(pitch['pz'])
#Interestingly, although in the end relatively unimportant, the standard deviation of this data ends up being around 1
#Therefore the z scores used here and the distances in the regression example are extremly similar
pitch['z_px'] = (pitch['px'] - mean_x)/pitch.px.std(ddof=0) 
pitch['z_pz'] = (pitch['pz'] - mean_z)/pitch.pz.std(ddof=0)

#The models probabilites are returned as a 2d list where each element has the probability of it being 
    

knn_prob = knn.predict_proba(pitch[['px', 'pz']])
knn_ps = []
for i in knn_prob:
    knn_ps.append(i[1])
    
pitch['knn_prob'] = knn_ps


nn_prob = mlp.predict_proba(pitch[['z_px', 'z_pz']])
nn_ps = []
for i in nn_prob:
    nn_ps.append(i[1])

pitch['nn_prob'] = nn_ps


pitch['prob'] = (pitch['knn_prob'] + pitch['nn_prob']) / 2
pitch['predict'] = pitch['prob'] > 0.5

In [65]:
print(str(np.sum(pitch['predict'] == pitch['call']) / len(pitch)) + "% accuracy for the combined model")
print("We don't really gain any accuracy combining these models, but I'm going to be using it from here on out.")

0.9149261132184425% accuracy for the combined model
We don't really gain any accuracy combining these models, but I'm going to be using it from here on out.


## Non-Location parameters

### Batter Handedness

I predicted batter handedness to be one of the most influential factors. However, to my surprise, the accuracy of our model was not significantly different for lefties or righties and adding handedness to the nearest neighbors model only improved it marginally.

In [76]:
ab = atbats[['ab_id', 'batter_id', 'p_throws', 'stand', 'pitcher_id', 'top', 'inning']]

#This is where it would have been nice to import the data into a SQL database
#But the benefit of not doing that is that this is much easier to share. My last project
#Was difficult to put online due to the database queries

#Joining the atbats and pitches on ab_id
p_ab = pd.merge(ab, pitch, on='ab_id')

#Splitting on left and right
left = p_ab.loc[p_ab['stand'] == "L"]
right = p_ab.loc[p_ab['stand'] == "R"]

def getAccuracy(predictions, actual):
    return (np.sum(predictions == actual) / (len(actual)))

print("Accuracy on left handed batters: " + str(getAccuracy(left['predict'], left['call'])) + "%")
print("Accuracy on right handed batters: " + str(getAccuracy(right['predict'], right['call'])) + "%")


Accuracy on left handed batters: 0.9146963997850618%
Accuracy on right handed batters: 0.915096390297216%


In [19]:
stand_float = []
for i in p_ab['stand']:
    if i == "L":
        stand_float.append(5)
    else:
        stand_float.append(0)
p_ab['stand_float'] = stand_float
in_full = p_ab[['px', 'pz', 'stand_float']].values
out_full = p_ab['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

knn = KNeighborsClassifier(n_neighbors = 95)
knn.fit(in_train, out_train)

#Predicting, this takes a while
test_predictions = knn.predict(in_test)
    
print(str(np.sum(test_predictions == out_test) / len(test_predictions)))
print("Considering our 91.45% accuracy for our base k-nearest neigbors model, a 91.73% accuracy isn't a big improvement")

0.9158440181744003
Considering our 91.45% accuracy for our base k-nearest neigbors model, a 91.73% accuracy isn't a big improvement


### Home/Away

While I don't expect there to be a significant difference in calls for the home or away team, finding a difference in this area would be problematic, so I decided to check. Fortuantely, based both on the lack of difference in the model's accuracy on home team at bats vs away team at bats as well as the model's lack of improvement when considering the half of the inning indicates that this is not something to worry about.

In [20]:
away_atbats = p_ab.loc[p_ab["top"] == True]
home_atbats = p_ab.loc[p_ab["top"] == False]

print("Original model accuracy on away team at bats: " + str(getAccuracy(away_atbats['predict'], away_atbats['call'])) + "%")
print("Original model accuracy on home team at bats: " + str(getAccuracy(home_atbats['predict'], home_atbats['call'])) + "%")


Original model accuracy on away team at bats: 0.9145569489976411%
Original model accuracy on home team at bats: 0.9153785283172989%


In [128]:
top_float = []
for i in p_ab['top']:
    if i:
        top_float.append(1)
    else:
        top_float.append(0)
p_ab['top_float'] = top_float
in_full = p_ab[['px', 'pz', 'top_float']].values
out_full = p_ab['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

knn = KNeighborsClassifier(n_neighbors = 95)
knn.fit(in_train, out_train)

#Predicting, this takes a while
test_predictions = knn.predict(in_test)
print(str(np.sum(test_predictions == out_test) / len(test_predictions)))
print("The 0.01% improvement isn't exactyly")

0.914470128357574


### Inning Number

atbats.head()



In [21]:

in_full = p_ab[['px', 'pz', 'inning']].values
out_full = p_ab['call']

in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

knn = KNeighborsClassifier(n_neighbors = 95)
knn.fit(in_train, out_train)

#Predicting, this takes a while
test_predictions = knn.predict(in_test)
print(str(np.sum(test_predictions == out_test) / len(test_predictions)))


0.9140168144586212


In [77]:
import os.path
from sklearn.neural_network import MLPClassifier
if not use_pickle or not os.path.isfile('mlp_inning_2.pkl'):
    mean_x = np.mean(p_ab['px'])
    mean_z = np.mean(p_ab['pz'])
    #Interestingly, although in the end relatively unimportant, the standard deviation of this data ends up being around 1
    #Therefore the z scores used here and the distances in the regression example are extremly similar
    p_ab['z_px'] = (p_ab['px'] - mean_x)/p_ab.px.std(ddof=0) 
    p_ab['z_pz'] = (p_ab['pz'] - mean_z)/p_ab.pz.std(ddof=0)

    #We split our data into training and test sets again (again)
    in_full = p_ab[['z_px', 'z_pz', 'inning']].values
    out_full = p_ab['call']

    in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

    #adam is reccomended for datasets with samples over 1000s
    #early stopping will set aside a validation set and stop when that is no longer improving
    mlp_inning = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(64, 64), random_state=1, early_stopping = True)

    mlp_inning.fit(in_train, out_train)
    if use_pickle:
        f=open('mlp_inning_2.pkl', 'wb')
        pickle.dump(mlp_inning,f)
        f.close()
else:
    f=open('mlp_inning_2.pkl', 'rb')
    mlp = pickle.load(f, encoding='latin1')
    f.close()

print(np.sum(mlp_inning.predict(in_test) == out_test) / len(out_test))

NameError: name 'mlp_inning' is not defined

## Umpire

It should not be surprising that the largest influence on the accuracy of balls and strikes is the umpire themselves. The umpires that have the highest accuracy 

In [78]:
#Joining pitch/atbat dataframe with the game csv on g_id (game id)
p_ab = pd.merge(p_ab, atbats[['ab_id', 'g_id']], on='ab_id')
p_ab = pd.merge(games[['umpire_HP', 'g_id']], p_ab, on='g_id')


p_ab['correct'] = p_ab['predict'] == p_ab['call']

In [79]:
umpires = p_ab.umpire_HP.unique()
umpires

array(['Mike Winters', 'Larry Vanover', 'Jeff Nelson', 'Dana DeMuth',
       'Gerry Davis', 'Jerry Layne', 'Ted Barrett', 'Dale Scott',
       'Joe West', 'Tim Welke', 'John Hirschbeck', 'Brian Gorman',
       'Gary Cederstrom', 'Bill Miller', 'Fieldin Culbreth', 'Ron Kulpa',
       'Laz Diaz', 'Ed Hickox', 'Dan Iassogna', 'Mark Carlson',
       'Eric Cooper', 'Doug Eddings', 'Brian Knight', 'Chris Guccione',
       'Paul Nauert', 'Phil Cuzzi', 'Hunter Wendelstedt',
       'Angel Hernandez', 'CB Bucknor', 'Kerwin Danley', 'Mike Everitt',
       'Sam Holbrook', 'Mike DiMuro', 'Rob Drake', 'Mark Wegner',
       'Jim Wolf', 'Jim Reynolds', 'Tony Randazzo', 'Bob Davidson',
       'Scott Barry', 'Lance Barksdale', 'Tim Timmons', 'Bill Welke',
       'Tripp Gibson III', 'Adam Hamari', 'Jerry Meals', 'Marty Foster',
       'Quinn Wolcott', 'Lance Barrett', 'Cory Blaser', 'Chris Conroy',
       'James Hoye', 'Paul Schrieber', 'Mike Estabrook', 'Vic Carapazza',
       'D.J. Reyburn', 'Will Litt

In [101]:
umpire_accuracy = []
umpire_count = []
for ump in umpires:
    ump_df =  p_ab.loc[p_ab.umpire_HP == ump]
    ump_acc = np.sum(ump_df['correct']) / len(ump_df)
    ump_len = len(ump_df)
    print(ump + ": " + str(ump_acc) + ", " + str(ump_len))
    umpire_accuracy.append(ump_acc)
    umpire_count.append(ump_len)
  
    


Mike Winters: 0.9064400715563506, 16770
Larry Vanover: 0.9091643582640813, 17328
Jeff Nelson: 0.9144529739879604, 18107
Dana DeMuth: 0.9105425672695192, 11335
Gerry Davis: 0.9061312759366416, 17109
Jerry Layne: 0.9072635255542092, 14029
Ted Barrett: 0.9089466825315882, 17649
Dale Scott: 0.9005248091603053, 8384
Joe West: 0.9098008792345488, 19335
Tim Welke: 0.8973192915270465, 4178
John Hirschbeck: 0.9036220871327254, 7896
Brian Gorman: 0.9149486556400408, 12757
Gary Cederstrom: 0.916853686572883, 18269
Bill Miller: 0.9134778099241225, 18319
Fieldin Culbreth: 0.9146922325354176, 16376
Ron Kulpa: 0.9133424098025867, 14690
Laz Diaz: 0.913472191446931, 18052
Ed Hickox: 0.9129495867153176, 13429
Dan Iassogna: 0.9077285377863412, 14013
Mark Carlson: 0.9154706746525031, 17698
Eric Cooper: 0.9199949840115368, 15949
Doug Eddings: 0.9085604283222042, 16623
Brian Knight: 0.9153033057294823, 14853
Chris Guccione: 0.9163556158805565, 17682
Paul Nauert: 0.9079788205221837, 16431
Phil Cuzzi: 0.91634

In [89]:
ump_df = p_ab.loc[p_ab.umpire_HP == umpires[0]]


In [104]:
#Training on just Joe West's data
joe_west_df =  p_ab.loc[p_ab.umpire_HP == "Joe West"]
if not use_pickle or not os.path.isfile('mlp_joe_west.pkl'):
    mean_x = np.mean(joe_west_df['px'])
    mean_z = np.mean(joe_west_df['pz'])
    #Interestingly, although in the end relatively unimportant, the standard deviation of this data ends up being around 1
    #Therefore the z scores used here and the distances in the regression example are extremly similar
    joe_west_df['z_px'] = (joe_west_df['px'] - mean_x)/joe_west_df.px.std(ddof=0) 
    joe_west_df['z_pz'] = (joe_west_df['pz'] - mean_z)/joe_west_df.pz.std(ddof=0)

    #We split our data into training and test sets again (again)
    in_full = joe_west_df[['z_px', 'z_pz']].values
    out_full = joe_west_df['call']

    in_train, in_test, out_train, out_test = train_test_split(in_full, out_full, test_size = 0.20)

    #adam is reccomended for datasets with samples over 1000s
    #early stopping will set aside a validation set and stop when that is no longer improving
    mlp_joe_west = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(64, 64), random_state=1, early_stopping = True)

    mlp_joe_west.fit(in_train, out_train)
    if use_pickle:
        f=open('mlp_joe_west.pkl', 'wb')
        pickle.dump(mlp_joe_west,f)
        f.close()
else:
    f=open('mlp_joe_west.pkl', 'rb')
    mlp = pickle.load(f, encoding='latin1')
    f.close()

print(np.sum(mlp_joe_west.predict(in_test) == out_test) / len(out_test))

0.9045771916214119


101